In [1]:
import torch

torch.__version__

'1.13.0.dev20220615'

In [3]:
import platform

if "macOS" in platform.platform():
    device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
else:
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device

device(type='mps')

In [4]:
text = '안녕하세요?'
seq = [1,2,3,4,5,6,7,8,9,1,1,2,3,11]

In [9]:
n_symbols = max(seq)+1
symbols_embedding_dim = 512

seq = torch.Tensor(seq).to(device=device, dtype=torch.long)
emb = torch.nn.Embedding(n_symbols, symbols_embedding_dim).to(device=device)

In [14]:
# https://github.com/sooftware/taKotron2/blob/master/text/__init__.py
import re
import unicodedata
from g2pk import G2p

CHOSUNGS = "".join([chr(_) for _ in range(0x1100, 0x1113)])
JOONGSUNGS = "".join([chr(_) for _ in range(0x1161, 0x1176)])
JONGSUNGS = "".join([chr(_) for _ in range(0x11A8, 0x11C3)])
SPECIALS = " ?!"

ALL_VOCABS = "".join([
    CHOSUNGS,
    JOONGSUNGS,
    JONGSUNGS,
    SPECIALS
])
VOCAB_DICT = {
    "_": 0,
    "~": 1,
}

for idx, v in enumerate(ALL_VOCABS):
    VOCAB_DICT[v] = idx + 2
    
symbols = VOCAB_DICT.keys()

g2p = G2p()


def normalize(text):
    text = unicodedata.normalize('NFKD', text)
    text = text.upper()
    regex = unicodedata.normalize('NFKD', r"[^ \u11A8-\u11FF\u1100-\u115E\u1161-\u11A7?!]")
    text = re.sub(regex, '', text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text


def tokenize(text, encoding: bool = True):
    tokens = list()

    for t in text:
        if encoding:
            tokens.append(VOCAB_DICT[t])
        else:
            tokens.append(t)

    if encoding:
        tokens.append(VOCAB_DICT['~'])
    else:
        tokens.append('~')

    return tokens


def text_to_sequence(text):
    text = g2p(text)
    text = normalize(text)
    tokens = tokenize(text, encoding=True)
    return tokens

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [15]:
text_to_sequence(text)

[13, 21, 45, 4, 27, 62, 20, 21, 11, 26, 13, 33, 70, 1]